In [8]:
from ModelClass import RegressionModel
from DatasetClass import Dataset, DatasetPt, DatasetMass
import matplotlib.pyplot as plt
import numpy as np
import itertools
from src.helpers import make_filter_slice
import tensorflow as tf
# import sys
# sys.path.append("/home/kvake/.local/lib/python3.11/site-packages")
import optuna
# sys.path.append("/home/kvake/.local/lib/python3.9/site-packages")  # Adjust if needed
import optuna_dashboard

In [9]:
erik_data = "/scratch/ucjf-atlas/htautau/SM_Htautau_R22/V02_skim_mva_01/*/*/*/*/*H125*.root"
patrik_data = "/scratch/ucjf-atlas/htautau/SM_Htautau_R22/V02_skim_mva_01/*/*/*/*/*Ztt*.root"

In [ ]:
dataset = Dataset()
# dataset.build_dataset()
# dataset.save_data()
dataset.load_data()

print(len(dataset.train_dataset))

#This block of code iterates through the dataset and extracts the pt values of the labels and stores them in a list
# data = [labels.numpy()[0] for features, labels in dataset.train_dataset.take(100000)]

# plt.hist(data, bins=100, range=(50, 130), histtype='step', label='pt distribution', density=False)
# plt.legend(loc='upper right')
# plt.title('pt distribution of the dataset')
# plt.xlabel('pt')
# plt.ylabel('Number of events')
# plt.show()

6523684


In [5]:
import optuna
from optuna_dashboard import run_server
from DatasetClass import DatasetPt
from ModelClass import RegressionModel
import threading

# ======================
# 1. Build/Load Dataset (Outside Optuna Loop for Efficiency)
# ======================
dataset = DatasetPt()
dataset.load_data()

print("Train dataset size:", len(dataset.train_dataset))

# ============================
# 2. Define Optuna Objective (Fast Training)
# ============================
def objective(trial):
    """
    Optuna objective function for hyperparameter search.
    """
    model = RegressionModel(
        dataset,
        n_layers=trial.suggest_int('n_layers', 3, 6),
        hidden_layer_size=2048,
        initial_learning_rate=trial.suggest_float('initial_learning_rate', 1e-3, 1e-2, log=True),
        n_epochs=80,
        activation_function  = 'relu',
        batch_size=trial.suggest_int('batch_size', 3000, 5000, step=1000),
        dropout_rate=0.2,
        weight_decay= 1e-5,
        optimizer='adamw',
        n_normalizer_samples = trial.suggest_int('n_normalizer_samples', 100, 10000, step=1000)
    )

    model.prepare_dataset()
    model.create_normalizer()
    model.build_model()
    model.train_model()

    val_loss, _ = model.model.evaluate(model.val_batch, verbose=0)
    return val_loss

# ========================
# 3. Run the Optuna Study
# ========================
study = optuna.create_study(storage="sqlite:///optuna.db", study_name="Higgs_analysis_jupyter_3", direction='minimize')

# ========================
# 4. Start `optuna-dashboard` in a Separate Thread
# ========================

def run_dashboard():
    server = run_server("sqlite:///optuna.db", host="0.0.0.0", port=8080)
    server.run()  # Run dashboard continuously in a separate thread

dashboard_thread = threading.Thread(target=run_dashboard)
dashboard_thread.daemon = True  # Ensures it stops when the script ends
dashboard_thread.start()

def run_optuna():
    study.optimize(objective, n_trials=10, n_jobs=1)

# Run Optuna in a separate thread so the dashboard can be accessed in real-time
optuna_thread = threading.Thread(target=run_optuna)
optuna_thread.start()

# ========================
# 5. Print Results
# ========================
optuna_thread.join()  # Wait for Optuna to finish before printing results
print("Number of finished trials:", len(study.trials))
print("Best trial:", study.best_trial.params)

# Dashboard will keep running in background


[I 2025-04-03 17:11:07,824] A new study created in RDB with name: Higgs_analysis_jupyter_3


Train dataset size: 6523684


Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8080/
Hit Ctrl-C to quit.

/singularity/ucjf/venv_tf_218/lib64/python3.9/site-packages/optuna/distributions.py:708: UserWarning: The distribution is specified by [100, 10000] and step=1000, but the range is not divisible by `step`. It will be replaced by [100, 9100].
  warnings.warn(


Batching datasets...


2025-04-03 17:11:08.566983: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2025-04-03 17:12:39.110896: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_91', 4 bytes spill stores, 4 bytes spill loads

2025-04-03 17:12:39.275325: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_91', 184 bytes spill stores, 184 bytes spill loads

2025-04-03 17:12:39.415236: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_91_0', 708 bytes spill stores, 420 bytes spill loads

2025-04-03 17:12:39.424055: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_91_0', 588 bytes spill stores, 776 bytes spill loads

2025-04-03 17:12:50.482889: I external/local_xla/xla/stream_

1305/1305 ━━━━━━━━━━━━━━━━━━━━ 102s 76ms/step - loss: 5241.6274 - mean_absolute_percentage_error: 27.8884 - mean_squared_error: 5241.6270 - val_loss: 229.0286 - val_mean_absolute_percentage_error: 6.5035 - val_mean_squared_error: 230.9575
Epoch 2/80
1305/1305 ━━━━━━━━━━━━━━━━━━━━ 93s 72ms/step - loss: 274.5453 - mean_absolute_percentage_error: 7.4102 - mean_squared_error: 274.5455 - val_loss: 202.0270 - val_mean_absolute_percentage_error: 5.9094 - val_mean_squared_error: 203.8420
Epoch 3/80
1305/1305 ━━━━━━━━━━━━━━━━━━━━ 89s 68ms/step - loss: 255.0307 - mean_absolute_percentage_error: 7.0486 - mean_squared_error: 255.0308 - val_loss: 192.8529 - val_mean_absolute_percentage_error: 5.7626 - val_mean_squared_error: 194.8601
Epoch 4/80
1305/1305 ━━━━━━━━━━━━━━━━━━━━ 93s 71ms/step - loss: 245.1629 - mean_absolute_percentage_error: 6.8532 - mean_squared_error: 245.1631 - val_loss: 186.8004 - val_mean_absolute_percentage_error: 5.5974 - val_mean_squared_error: 188.6035
Epoch 5/80
1305/1305 ━━

KeyboardInterrupt: 

1305/1305 ━━━━━━━━━━━━━━━━━━━━ 167s 84ms/step - loss: 207.5097 - mean_absolute_percentage_error: 6.1661 - mean_squared_error: 207.5098 - val_loss: 168.4962 - val_mean_absolute_percentage_error: 5.3295 - val_mean_squared_error: 170.2690
Epoch 29/80
  65/1305 ━━━━━━━━━━━━━━━━━━━━ 1:16 62ms/step - loss: 214.3988 - mean_absolute_percentage_error: 6.2126 - mean_squared_error: 214.3988

[W 2025-04-03 18:07:35,334] Trial 0 failed with parameters: {'n_layers': 4, 'initial_learning_rate': 0.003301989989224651, 'batch_size': 5000, 'n_normalizer_samples': 8100} because of the following error: ZMQError('Socket operation on non-socket').
Traceback (most recent call last):
  File "/singularity/ucjf/venv_tf_218/lib64/python3.9/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_4129364/389827429.py", line 39, in objective
    model.train_model()
  File "/home/kvake/HiggsAnalysisBc/ModelClass.py", line 147, in train_model
    history = self.model.fit(
  File "/singularity/ucjf/venv_tf_218/lib64/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/singularity/ucjf/venv_tf_218/lib64/python3.9/site-packages/ipykernel/iostream.py", line 694, in write
    self._schedule_flush()
  File "/singularity/ucjf/venv_tf_218/lib64